In [1]:
import pandas as pd
import numpy as np
import tabula
import re
pd.set_option('display.max_rows', 500)

In [2]:
dft_2021 = tabula.read_pdf('2021 Blog.pdf', pages='all', stream=True, pandas_options={'header': None})
dft_2022 = tabula.read_pdf('2022 12 Patch Day Blog V9.0.pdf', pages='all', stream=True, pandas_options={'header': None})
dft_2023 = tabula.read_pdf('2023 12 Patch Day Blog V2.0 (1).pdf', pages='all', stream=True, pandas_options={'header': None})

Jan 14, 2025 7:35:05 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 14, 2025 7:35:06 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 14, 2025 7:35:07 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 14, 2025 7:35:07 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 14, 2025 7:35:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Jan 14, 2025 7:35:08 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>


In [3]:
def newHeader(xdf):
    data_list = []
    header = xdf[0].iloc[0]
    xdf[0].columns = header
    xdf[0] = xdf[0].drop([0])
    xdf[0]["cve_id"] = xdf[0]["Title"].str.extract(r'(CVE-....-\d+)')
    for i in range(0, len(xdf)):
        if i == 0:
            data = xdf[i].iloc[1:]
        else:
            data = xdf[i]
        if data.shape[1] == header.shape[0]:
            data.columns = header
            xdf[i]["cve_id"] = xdf[i]["Title"].str.extract(r'(CVE-....-\d+)')
            data_list.append(data)
    return data_list

In [4]:
dftt_2021 = newHeader(dft_2021)
dftt_2022 = newHeader(dft_2022)
dftt_2023 = newHeader(dft_2023)

In [5]:
sap_2021 = pd.concat(dftt_2021)
sap_2022 = pd.concat(dftt_2022)
sap_2023 = pd.concat(dftt_2023)

In [6]:
# Pandas merge rows in a column 'Title'
# https://stackoverflow.com/questions/64141451/pandas-merge-rows-in-a-column
def mergeTitle(ydf):
    ydf['Title'] = ydf['Title'].astype(str)
    blocks = ydf['CVSS'].notna().cumsum()
    agg_dict = {col:' '.join if col=='Title' else 'first' for col in ydf}
    df_t = ydf.groupby(blocks).agg(agg_dict).reset_index(drop=True)
    df_t.dropna(inplace=True)
    df_t = df_t[(df_t['Note#'] != 'Note#')]
    df_t['Note#'] = df_t['Note#'].astype(int)
    df_t.reset_index(drop=True, inplace=True)
    return df_t

In [7]:
sap_2021_notes = mergeTitle(sap_2021)
sap_2022_notes = mergeTitle(sap_2022)
sap_2023_notes = mergeTitle(sap_2023)

In [8]:
#sap_2023_notes

In [9]:
sap_2024_all = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/bulletin-2024.html',
                            flavor='html5lib')

In [10]:
def ds_sap_24(x):
    sap_2024_ls = []
    for mes in x:
        mes.rename(columns={'Severity': 'Priority'}, inplace=True)
        sap_2024_ls.append(mes)
    sap_2024 = pd.concat(sap_2024_ls, ignore_index=True)
    sap_2024["cve_id"] = sap_2024["Title"].str.extract(r'(CVE-....-\d+)')
    return sap_2024

In [11]:
sap_2024_df = ds_sap_24(sap_2024_all)

In [12]:
sap_2024_notes = sap_2024_df.dropna()
#sap_2024_notes = pd.read_csv('sap_notes_2024.csv')

In [13]:
sap_202501 = pd.read_html('https://support.sap.com/en/my-support/knowledge-base/security-notes-news/january-2025.html',
                 flavor='html5lib')

In [17]:
def etData(x):
    _df = pd.DataFrame(x[0], columns=['Note#', 'Title', 'Severity', 'CVSS'])
    _df.rename(columns={'Severity': 'Priority'}, inplace=True)
    _df["cve_id"] = _df["Title"].str.extract(r'(CVE-....-\d+)')
    return _df

In [18]:
sap_202501_df = etData(sap_202501)

In [20]:
sap_202501_notes = sap_202501_df.dropna()

In [24]:
sap_202501_notes.replace({'Priority': 'Critical'}, {'Priority': 'Hot News'}, inplace=True)

In [25]:
sap_202501_notes

,Note#,Title,Priority,CVSS,cve_id
0,3537476,[CVE-2025-0070] Improper Authentication in SAP...,Hot News,9.9,CVE-2025-0070
1,3550708,[CVE-2025-0066] Information Disclosure vulnera...,Hot News,9.9,CVE-2025-0066
2,3550816,[CVE-2025-0063] SQL Injection vulnerability in...,High,8.8,CVE-2025-0063
3,3474398,[CVE-2025-0061] Multiple vulnerabilities in SA...,High,8.7,CVE-2025-0061
4,3542533,[CVE-2025-0069] DLL Hijacking vulnerability in...,High,7.8,CVE-2025-0069
5,3542698,[CVE-2025-0058] Information Disclosure vulnera...,Medium,6.5,CVE-2025-0058
6,3540108,[CVE-2025-0067] Missing Authorization check in...,Medium,6.3,CVE-2025-0067
7,3472837,[CVE-2025-0055] Information Disclosure vulnera...,Medium,6.0,CVE-2025-0055
8,3502459,[CVE-2025-0056] Information Disclosure vulnera...,Medium,6.0,CVE-2025-0056
9,3503138,[CVE-2025-0059] Information Disclosure vulnera...,Medium,6.0,CVE-2025-0059


In [26]:
# start extract cve_ids string for terminal commands
# Mejorar y crear un file para flag del commando.
l_sap_cve = sap_202501_notes.cve_id.to_list()

In [27]:
clean_sap_cve = [x for x in l_sap_cve if str(x) != 'nan']

In [28]:
#cleanedList

In [31]:
string_list = [str(element) for element in clean_sap_cve]
delimiter = " "
result_string_cve = delimiter.join(string_list)

In [32]:
result_string_cve

'CVE-2025-0070 CVE-2025-0066 CVE-2025-0063 CVE-2025-0061 CVE-2025-0069 CVE-2025-0058 CVE-2025-0067 CVE-2025-0055 CVE-2025-0056 CVE-2025-0059 CVE-2025-0053 CVE-2025-0057 CVE-2025-0068 CVE-2024-29131'

In [ ]:
# end extract cve_ids

In [33]:
### Crear DataFrame de output Sploitscan
def dataframeSplotscan(file_json):
    data = pd.DataFrame(columns = ['cve_id', 'dateUpdated', 'descriptions', 'product_l', 'epss_l', 'percentile', 'priority_l', 'cweId'])
    dict_list = []
    sap_sp = pd.read_json(f'{file_json}', typ='series')
    for i in sap_sp:
        if 'problemTypes' in i['CVE Data']['containers']['cna'].keys():            
            if 'cweId' in i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0].keys():
                cweId = i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0]['cweId']
            else:
                cweId = i['CVE Data']['containers']['cna']['problemTypes'][0]['descriptions'][0]['description']
        else:
            cweId = None
        if len(i['EPSS Data']['data']) == 1:
            epss_l = i['EPSS Data']['data'][0]['epss']
            percentile = i['EPSS Data']['data'][0]['percentile']
        else:
            epss_l = None
            percentile = None
        if 'datePublished' in i['CVE Data']['cveMetadata']:
            datePublished = i['CVE Data']['cveMetadata']['datePublished']
        else:
            datePublished = None
        if 'references' in i['CVE Data']['containers']['cna'].keys():            
            note_id = re.findall('[2,3]{1}[0-9]{6}', str(i['CVE Data']['containers']['cna']['references'][0]['url']))
        else:
            note_id = None
        row_list = {'cve_id': i['CVE Data']['cveMetadata']['cveId'],
                    #'datePublished': i['CVE Data']['cveMetadata']['datePublished'],
                    'datePublished': datePublished,
                    'dateUpdated': i['CVE Data']['cveMetadata']['dateUpdated'],
                    'descriptions': i['CVE Data']['containers']['cna']['descriptions'][0]['value'],
                    'product_l': i['CVE Data']['containers']['cna']['affected'][0]['product'],
                    #'epss_l': i['EPSS Data']['data'][0]['epss'],
                    #'benchmark': i['VulnCheck Data']['_benchmark'],
                    'epss_l': epss_l,
                    'percentile': percentile,
                    'priority_l': i['Priority']['Priority'],
                    'cweId': cweId,
                    'note_id': str(note_id)}
        dict_list.append(row_list)

    data = pd.DataFrame.from_dict(dict_list)
    return(data)

In [34]:
#especial 2021 sap notes
#sp_sap_cve_2021 = dataframeSplotscan('sap_history_data_curate/sp_2021_last.json')

In [35]:
sp_sap_cve_2021 = dataframeSplotscan('sap_history_data_curate/aws_2021.json')
sp_sap_cve_2021.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2022 = dataframeSplotscan('sap_history_data_curate/aws_2022.json')
sp_sap_cve_2022.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2023 = dataframeSplotscan('sap_history_data_curate/aws_2023.json')
sp_sap_cve_2023.drop_duplicates(subset=['cve_id'], inplace=True)
#sp_sap_cve_2024 = dataframeSplotscan('sap_history_data_curate/sp_2024_last.json')
sp_sap_cve_2024 = dataframeSplotscan('sap_history_data_curate/aws_2024.json')
sp_sap_cve_2024.drop_duplicates(subset=['cve_id'], inplace=True)
sp_sap_cve_2025 = dataframeSplotscan('sap_history_data_curate/aws_202501.json')
sp_sap_cve_2025.drop_duplicates(subset=['cve_id'], inplace=True)

In [59]:
#sp_sap_cve_2024

In [36]:
cp_sap_cve_2021 = pd.read_csv('sap_history_data_curate/aws_2021.csv')
cp_sap_cve_2022 = pd.read_csv('sap_history_data_curate/aws_2022.csv')
cp_sap_cve_2023 = pd.read_csv('sap_history_data_curate/aws_2023.csv')
cp_sap_cve_2024 = pd.read_csv('sap_history_data_curate/aws_2024.csv')
cp_sap_cve_2025 = pd.read_csv('sap_history_data_curate/aws_202501.csv')

In [37]:
cp_sap_cve_2021.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2022.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2023.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2024.drop_duplicates(subset=['cve_id'], inplace=True)
cp_sap_cve_2025.drop_duplicates(subset=['cve_id'], inplace=True)

In [38]:
sap_cve_2023 = sap_2023_notes.merge(sp_sap_cve_2023, on=['cve_id'])
sap_cve_2023 = sap_cve_2023.merge(cp_sap_cve_2023, on=['cve_id'])
sap_cve_2023['sap_note_year'] = '2023'

In [39]:
sap_cve_2022 = sp_sap_cve_2022.merge(sap_2022_notes, on=['cve_id'])
sap_cve_2022 = sap_cve_2022.merge(cp_sap_cve_2022, on=['cve_id'])
sap_cve_2022['sap_note_year'] = '2022'

In [40]:
sap_cve_2021 = sp_sap_cve_2021.merge(sap_2021_notes, on=['cve_id'])
sap_cve_2021 = sap_cve_2021.merge(cp_sap_cve_2021, on=['cve_id'])
sap_cve_2021['sap_note_year'] = '2021'

In [41]:
sap_cve_2024 = sp_sap_cve_2024.merge(sap_2024_notes, on=['cve_id'])
sap_cve_2024 = sap_cve_2024.merge(cp_sap_cve_2024, on=['cve_id'])
sap_cve_2024['sap_note_year'] = '2024'

In [43]:
sap_cve_2025 = sp_sap_cve_2025.merge(sap_202501_notes, on=['cve_id'])
sap_cve_2025 = sap_cve_2025.merge(cp_sap_cve_2025, on=['cve_id'])
sap_cve_2025['sap_note_year'] = '2025'

In [44]:
sap_cve_df = pd.concat([sap_cve_2021,sap_cve_2022,sap_cve_2023,sap_cve_2024,sap_cve_2025], ignore_index=True)

In [45]:
sap_cve_df.loc[(sap_cve_df['Priority'] == 'Hot') | (sap_cve_df['Priority'] == 'HotNews') | (sap_cve_df['Priority'] == 'Very High'), 'Priority'] = 'Hot News'
#sap_cve_df.loc[(sap_cve_df['Priority'] == 'Hot') | (sap_cve_df['Priority'] == 'HotNews'), 'Priority'] = 'Hot News'

sap_cve_df.loc[(sap_cve_df['cweId'] == 'Cross-Site Scripting') | (sap_cve_df['cweId'] == 'Cross Site Scripting') | (sap_cve_df['cweId'] == "Cross-Site Scripting (XSS)"), 'cweId'] = "CWE-79"

sap_cve_df.loc[(sap_cve_df['cweId'] == 'Missing Authorization check') | (sap_cve_df['cweId'] == 'Missing Authorization') | (sap_cve_df['cweId'] == 'Missing Authorization Check'), 'cweId'] = "CWE-862"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21484'),'cweId'] = "CWE-863"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2023-30533'),'cweId'] = "CWE-1321"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2022-35737'),'cweId'] = "CWE-129"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2023-44487'),'cweId'] = "CWE-400"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6308'),'cweId'] = "CWE-918"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6207'),'cweId'] = "CWE-306"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33690'),'cweId'] = "CWE-918"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38163'),'cweId'] = "CWE-78"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44235'),'cweId'] = "CWE-78"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-37531'),'cweId'] = "CWE-78"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33663'),'cweId'] = "CWE-74"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2024-33007'),'cweId'] = "CWE-79"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27608'),'cweId'] = "CWE-428"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27635'),'cweId'] = "CWE-112"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27635'),'cweId'] = "CWE-112"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27617'),'cweId'] = "CWE-112"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-40499'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27611'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21466'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27602'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44231'),'cweId'] = "CWE-94"
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21480'),'cweId'] = "CWE-94"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-10683'),'cweId'] = "CWE-611"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21444'),'cweId'] = "CWE-1021"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2019-17495'),'cweId'] = "CWE-352"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44151'),'cweId'] = "CWE-330"

sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2013-3587'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2019-0388'),'cweId'] = 'CWE-290'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-26816'),'cweId'] = 'CWE-312'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6215'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6224'),'cweId'] = 'CWE-532'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21445'),'cweId'] = 'CWE-444'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21449'),'cweId'] = 'CWE-119'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21465'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21469'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21470'),'cweId'] = 'CWE-611'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21472'),'cweId'] = 'CWE-306'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21474'),'cweId'] = 'CWE-326'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21475'),'cweId'] = 'CWE-22'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21476'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21477'),'cweId'] = 'CWE-94'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21478'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21488'),'cweId'] = 'CWE-502'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21491'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27610'),'cweId'] = 'CWE-287'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27612'),'cweId'] = 'CWE-601'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-27638'),'cweId'] = 'CWE-20'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33672'),'cweId'] = 'CWE-116'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33676'),'cweId'] = 'CWE-862'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33685'),'cweId'] = 'CWE-22'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33687'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-33688'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38150'),'cweId'] = 'CWE-312'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38176'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-38177'),'cweId'] = 'CWE-476'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-40497'),'cweId'] = 'CWE-668'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-42064'),'cweId'] = 'CWE-89'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-42068'),'cweId'] = 'CWE-20'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-44232'),'cweId'] = 'CWE-22'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2023-0215'),'cweId'] = 'CWE-416'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-6369'),'cweId'] = 'CWE-798'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2020-13936'),'cweId'] = 'CWE-94'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21446'),'cweId'] = 'CWE-400'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21482'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21483'),'cweId'] = 'CWE-200'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2021-21485'),'cweId'] = 'CWE-200'
#sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2024-29415'),'cweId'] = 'CWE-941'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2024-47593'),'cweId'] = 'CWE-524'
sap_cve_df.loc[(sap_cve_df['cve_id'] == 'CVE-2022-26104'),'cweId'] = 'CWE-862'

In [46]:
sap_cve_df.to_csv('sap_cve_2025_aws.csv', index=False)

In [47]:
sap_cve_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cve_id         702 non-null    object 
 1   datePublished  702 non-null    object 
 2   dateUpdated    702 non-null    object 
 3   descriptions   702 non-null    object 
 4   product_l      702 non-null    object 
 5   epss_l         702 non-null    object 
 6   percentile     702 non-null    object 
 7   priority_l     702 non-null    object 
 8   cweId          702 non-null    object 
 9   note_id        702 non-null    object 
 10  Note#          702 non-null    int64  
 11  Title          702 non-null    object 
 12  Priority       702 non-null    object 
 13  CVSS           702 non-null    object 
 14  priority       702 non-null    object 
 15  epss           702 non-null    float64
 16  cvss           702 non-null    float64
 17  cvss_version   702 non-null    object 
 18  cvss_sever

In [48]:
sap_cve_df.loc[(sap_cve_df['Priority'] == 'Medium')]['Priority'].count()

421

In [49]:
sap_cve_df.loc[660]

cve_id                                              CVE-2024-45284
datePublished                             2024-09-10T04:57:24.442Z
dateUpdated                               2024-09-10T13:45:15.831Z
descriptions     An authenticated attacker with high privilege ...
product_l                 SAP Student Life Cycle Management (SLcM)
epss_l                                                 0.000430000
percentile                                             0.111570000
priority_l                                                       D
cweId                                                      CWE-862
note_id                                                ['2256627']
Note#                                                      2256627
Title            [CVE-2024-45284] Missing authorization check i...
Priority                                                       Low
CVSS                                                           2.7
priority                                                Priori